In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import xml.etree.ElementTree as ET

# Загрузка данных из XML файла
tree = ET.parse('отчетность.xml')
root = tree.getroot()

data = []
for company in root.findall('Компания'):
    company_data = {
        'Название': company.get('Название'),
        'Отчетная_дата': int(company.find('Показатели').get('Отчетная_дата')),
        'Сумма_активов': int(company.find('Показатели').get('Сумма_активов')),
        'Сумма_обязательств': int(company.find('Показатели').get('Сумма_обязательств')),
        'Количество_выданных_микрозаймов': int(company.find('Показатели').get('Количество_выданных_микрозаймов')),
        'Сумма_выданных_микрозаймов': int(company.find('Показатели').get('Сумма_выданных_микрозаймов'))
    }
    data.append(company_data)

df = pd.DataFrame(data)

# Разделение данных на признаки и целевые переменные
X = df[['Сумма_активов', 'Сумма_обязательств']]
y_count = df['Количество_выданных_микрозаймов']
y_amount = df['Сумма_выданных_микрозаймов']

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Обучение модели случайного леса для количества выданных микрозаймов
rf_count = RandomForestRegressor()
rf_count.fit(X_scaled, y_count)

# Обучение модели случайного леса для суммы выданных микрозаймов
rf_amount = RandomForestRegressor()
rf_amount.fit(X_scaled, y_amount)

# Создание DataFrame для прогнозирования на 3 отчетных периода вперед
future_periods = 3
future_data = []
for i in range(future_periods):
    for index, row in df.iterrows():
        future_company_data = {
            'Название': row['Название'],
            'Отчетная_дата': row['Отчетная_дата'] + i + 1,
            'Сумма_активов': None,
            'Сумма_обязательств': None,
            'Количество_выданных_микрозаймов': None,
            'Сумма_выданных_микрозаймов': None
        }
        future_data.append(future_company_data)

future_df = pd.DataFrame(future_data)

# Прогнозирование количества и суммы выданных микрозаймов на 3 отчетных периода вперед для каждой организации
X_future = future_df[['Сумма_активов', 'Сумма_обязательств']]
X_future_scaled = scaler.transform(X_future)

future_df['Прогноз_количества'] = rf_count.predict(X_future_scaled)
future_df['Прогноз_суммы'] = rf_amount.predict(X_future_scaled)

# Обновление данных в DataFrame с учетом прогнозов
for index, row in future_df.iterrows():
    org_name = row['Название']
    report_date = row['Отчетная_дата']
    original_data = df[(df['Название'] == org_name) & (df['Отчетная_дата'] == report_date)]
    if not original_data.empty:
        future_df.loc[(future_df['Название'] == org_name) & (future_df['Отчетная_дата'] == report_date), 'Сумма_активов'] = original_data['Сумма_активов'].values[0]
        future_df.loc[(future_df['Название'] == org_name) & (future_df['Отчетная_дата'] == report_date), 'Сумма_обязательств'] = original_data['Сумма_обязательств'].values[0]
        future_df.loc[(future_df['Название'] == org_name) & (future_df['Отчетная_дата'] == report_date), 'Количество_выданных_микрозаймов'] = original_data['Количество_выданных_микрозаймов'].values[0]
        future_df.loc[(future_df['Название'] == org_name) & (future_df['Отчетная_дата'] == report_date), 'Сумма_выданных_микрозаймов'] = original_data['Сумма_выданных_микрозаймов'].values[0]

# Сохранение результатов прогнозирования в новый файл
future_df.to_csv('прогнозы.csv', index=False)